# Hashing
Message -> Hash Function -> Hash Value(number)

variable-length input -> fixed-length output

## Hash function properties
- deterministic behaviour - for given input produces the same output
- fixed-length hash values 
- avalanche effect - when small difference in message results in large differences between hash values

eg. `hash()` function used to hash keys in Python dicts. For cryptographic hash function additional properties must be met.


## Cryptographic hash function properties 
- One-way function property - must be difficult to identify input from given output
- Weak collision resistance - given one message it's infeasible to identify second message that computes to the same hash value
- Strong collision resistance - it's infeasible to find any collision at all


In [ ]:
print(hash("dupa"))
print(hash("dupa"))
print(hash("dupa2"))

## Cryptographic hashing in Python

In [ ]:
import hashlib

# list of all hash algorithms
print(hashlib.algorithms_available)

# list of hash algorithms available on all platforms
print(hashlib.algorithms_guaranteed)

`MD5` and `SHA1` are no longer suitable for data integrity.

Use `SHA2` (standard) or `SHA3` (new standard) or `Blake` (fast). Most common is `SHA256`.


In [ ]:
import hashlib

# Python 3 strings saved in unicode code points (UTF-8)
# Hash function argument must be bytes; strings must be encoded to become bytes
hash1 = hashlib.sha256(b'duuupa')
hash2 = hashlib.sha256('duuupa'.encode())
print(hash1.digest_size, 'bytes')

# Hash value in str
print(hash1.hexdigest())
print(hash2.hexdigest())

# Hash value in bytes
print(hash1.digest())
print(hash2.digest())


In [ ]:
from hashlib import sha256

# Chunked hash generation using update()
many = sha256()
many.update(b'm')
many.update(b'e')
many.update(b's')
many.update(b's')
print(many.hexdigest())

print(sha256(b'mess').hexdigest())

## Checksum functions
Checksums (eg. CRC, Adler-32) are fast and have insufficient collision resistance - can be used for error detection.

Hash functions (SHA2 family, SHA3 family) are slower and have sufficient collision resistance - can be used for testing data integrity.

In [ ]:
import zlib

# CRC checksum collision
print(zlib.crc32(b'gnu'))
print(zlib.crc32(b'codding'))

# Adler-32 checksum no collision
print(zlib.adler32(b'gnu'))
print(zlib.adler32(b'codding'))


# Keyed hashing
## Data authentication
Data authentication (who authored the change?) - requires __key__ and a __keyed hash function__

### Key generation
Key can be in form of:
- random number - sequence of random numbers
- passphrase - sequence of random words

### Random number
Keys that are hard to remember

Use `secrets` module. Do not use `random` module.

In [ ]:
import os

# random secret generation - 16 bytes
print(os.urandom(16))

In [ ]:

from secrets import token_bytes, token_hex, token_urlsafe

# random secret generation - 16 bytes
print(token_bytes(16))
print(token_hex(16))
print(token_urlsafe(16))

### Passphrases
Keys that are easy to remember

In [ ]:
import secrets
from pathlib import Path

words = Path('wordlist.txt').read_text().splitlines()
passphrase = ' '.join(secrets.choice(words) for i in range(4))
print(passphrase)

### Keyed hashing
__Keyed hash functions__ use _key and message_ to produce _hash value_. The same message with different key is hashed to different hash value. Only some functions can do that by default like `blake2b`.

In [ ]:
from hashlib import blake2b

m = b'message'
x = b'key x'
y = b'key y'

print(blake2b(m, key=x).hexdigest())
print(blake2b(m, key=y).hexdigest())

## HMAC functions
Hash-based Message Authentication Code.

HMAC functions allow any generic hashing function to become keyed hash functions. It has 3 inputs:
- message
- key
- hash function

In [ ]:
import hashlib
import hmac

xx = hmac.new(key=b'key', msg=b'message', digestmod=hashlib.sha3_256)
print(xx.name)  # protocol name prefixed with HMAC
print(xx.hexdigest())

### Data authentication between parties

In [ ]:
import hashlib
import hmac
import json
hmac_sha256 = hmac.new(b'shared_key', digestmod=hashlib.sha256)
message = b'from Bob to Alice'
hmac_sha256.update(message)
hash_value = hmac_sha256.hexdigest()
authenticated_msg = {
    'message': list(message),
    'hash_value': hash_value, 
    }
outbound_msg_to_alice = json.dumps(authenticated_msg)

In [ ]:
import hashlib
import hmac
import json
authenticated_msg = json.loads(inbound_msg_from_bob)
message = bytes(authenticated_msg['message'])
hmac_sha256 = hmac.new(b'shared_key', digestmod=hashlib.sha256)
hmac_sha256.update(message)
hash_value = hmac_sha256.hexdigest()
if hash_value == authenticated_msg['hash_value']:
    print('trust message')

## Timing attacks
String comparison of 2 hash values is faster if they are different (evaluates to False faster). Attacker can measure response time to invalid hash (_side channel attack_).

To mitigate this problem use length-constant time or random time. Always use `compare_digest()` for hashes.


In [ ]:
from hmac import compare_digest

compare_digest('abc', 'abcd')

# Symmetric encryption
Encryption - process of obfuscating plaintext into ciphertext using cipher (encryption algorithm) together with key. Encryption ensures confidentiality.

Decryption - reverse process

Fernet guarantees that a message encrypted using it cannot be manipulated or read without the key. Fernet is an implementation of symmetric (also known as “secret key”) authenticated cryptography. Fernet also has support for implementing key rotation via MultiFernet.

In [ ]:
from cryptography.fernet import Fernet
key = Fernet.generate_key()  # random 32 byte key in bytes format
print(key)
fernet = Fernet(key)  # general purpose class initialized with key

# token - combined ciphertext HMAC hash value from that ciphertext (confidentiality + message authenticity???)
token = fernet.encrypt(b'duuupa')
token

## Key rotation
Key rotation - retire one key with another. It means decrypting all ciphertext with old key and reencrypting them with new key

In [ ]:
from cryptography.fernet import Fernet, MultiFernet

# Encrypting with old key
old_key = Fernet.generate_key()
old_fernet = Fernet(old_key)
old_token1 = old_fernet.encrypt(b'dupa')
old_token2 = old_fernet.encrypt(b'krowa')

# Creating new key
new_key = Fernet.generate_key()
new_fernet = Fernet(new_key)

multi_fernet = MultiFernet([new_fernet, old_fernet])
# List of tokens encrypted with old key
old_tokens = [old_token1, old_token2]
print(old_tokens)
# Decrypting old tokens and reencrypting them with new key
new_tokens = [multi_fernet.rotate(t) for t in old_tokens]
print(new_tokens)

#replace_old_tokens(new_tokens)
#replace_old_key_with_new_key(new_key)
#del old_key

# Decrypt after rotation
for new_token in new_tokens:
    plaintext = new_fernet.decrypt(new_token)
    print(plaintext)

### Categories of encryption algorithms
- block ciphers (AES - 128, 192, 256) - encrypts fixed-length blocks
- stream ciphers (ChaCha) - processes stream of bytes

### Symmetric encryption modes:
- ECB (Electronic CodeBook) - for less than block-length plaintext fills in with padding
- CBC (Cipher Block Chaining) - with initialization vector (IV)
- GCM (Galois Counter Mode)
- others

In [ ]:
# ECB example
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes


key = b'key must be 128, 196 or 256 bits'
cipher = Cipher(
    algorithms.AES(key),  # AES cipher
    modes.ECB(),  # ECB mode
    backend=default_backend()  # OpenSSL
    )

encryptor = cipher.encryptor()
plaintext = b'block size = 128'  # signle block of plaintext

ciphertext = encryptor.update(plaintext) + encryptor.finalize()  # single block of ciphertext
print(ciphertext)

### Summary
- ecryption ensures confidentiality
- symmetric encryption algorithms use the same key for encryption and decryption

# Asymmetric encryption

## RSA public-key encryption
 
RSA - public-key cryptosystem that involves 4 steps:
1. key generation
1. key distribution
1. encryption
1. decryption


Create private key:

`openssl genpkey -algorithm RSA -out private_key.pem -pkeyopt rsa_keygen_bits:2048`

Create public key:

`openssl rsa -pubout -in private_key.pem -out public_key.pem`

 
Public-key cryptography allows:
- encrypting with public key and only private key can decrypt - ensures data confidentiality
- encrypting with private key and only public key can decrypt - ensures data authenticity
 
 
## Nonrepudiation
When system prevents a participant from denying their actions. Assymentric encryption alone
 
Assymentric encryption alone does not ensure nonrepudiation because eg.:
- anyone with public key can create message and claim it's from someone else
- anyone who received message encrypted with private key can say he is the author of the message
 
In order to achieve nonrepudiation a digital signature is needed.
 
## RSA Digital signatures
Digital signature allows __anyone__ to check:
- who sent the message?
- has the message been modified in transit?
 
Digital signature:
- is unique to signer
- can be used to legally bind the signer to a contract
- is difficult to forge
 
Digital signature combines hashing with public-key encryption.
 
Sender sends message hash encrypted with private key along with message itself
Receiver can decrypt the hash with public key and compare it with computed hash of a message
 
RSA digital signing uses different padding scheme than RSA encryption.
 
## Eclyptic-curve digital signatures
Cryptosystem similar to RSA with following characteristics:
- uses key-pair for signing data and verifying signatures
- private key cannot decrypt what public key encrypted
- faster signing data and verifying signatures than RSA
- less lines of code to use
 
 
## Summary
- Hashing ensures data integrity and data authentication.
- Encryption ensures confidentiality.
- Digital signatures ensure nonrepudiation

# TLS - Transport Layer Security
- ensures data integrity, data authentication, confidentiality, and nonrepudiation
- point-to-point client/server protocol
 
## TLS handshake
Typically client initiates handshake(s) with server. Handshake objective is to perform:
- cipher suite negotiation
- key exchange
- server authentication
 
### Cipher suite negotiation
Client and server must first agree on common set of algorithms known as cipher suite. 
Cipher suite defines encryption and hashing algorithms.
Each TLS version defines different cipher suites. 


TLS 1.2 defines 37 cipher suites


TLS 1.3 defines 5 cipher suites:
- TLS_AES_128_CCM_8_SHA256
- TLS_AES_128_CCM_SHA256
- TLS_AES_128_GCM_SHA256
- TLS_AES_256_GCM_SHA384
- TLS_CHACHA20_POLY1305_SHA256


TLS_AES_128_GCM_SHA256:
- TLS_- common prefix
- AES_128_GCM - symmetric encryption with 128bit key in GCM mode
- _SHA256 - hashing


TLS ensures confidentiality with symmetric encryption (it's more efficient than assymetric encryption).
 
### Key exchange
Client and server must exchange a key that will be used with cipher suite for symmetric encryption.

In TLS 1.2 key-distribution problem is solved by Diffie-Hellman key exchange or STATIC RSA key-exchange method (worse)

In TLS 1.3 key-distribution problem is solved by Diffie-Hellman key exchange (Perfect Forward Secrecy)

Diffie-Hellman key-exchange is one-roundtrip key-exchange algorithm that results in both nodes independently computing shared secret key for symmetric encryption.
 
 
 
### Server authentication
Public-key encryption is used for sever authentication
 
## Django
 
## Gunicorn